In [1]:
import pandas as pd
import numpy as np

In [2]:
CLIENT_ID = 'GKUAF244OPVYY01Z33OBZRVVQ5XFFLMKSKR5PNF425QFG0CU' 
CLIENT_SECRET = 'CENIDR0OMZIIKAPI0CT145ZYU3J2OQUKFLRMRD4BLJSJRQZI' 
VERSION = '20180605' 

In [3]:
import numpy as np

from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

import json
from pandas.io.json import json_normalize

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  55.38 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  37.33 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  40.81 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  49.49 MB/s


In [4]:
#get coordinates of New York

address = 'New York, New York'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York are 40.7127281, -74.0060152.


In [5]:
#get venues in New York

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=GKUAF244OPVYY01Z33OBZRVVQ5XFFLMKSKR5PNF425QFG0CU&client_secret=CENIDR0OMZIIKAPI0CT145ZYU3J2OQUKFLRMRD4BLJSJRQZI&v=20180605&ll=40.7127281,-74.0060152&radius=500&limit=100'

In [6]:
import requests 
results = requests.get(url).json()

In [7]:
# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [8]:
#search for dog parks in NYC

search_query = 'Dog Park'
radius = 4000

In [9]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GKUAF244OPVYY01Z33OBZRVVQ5XFFLMKSKR5PNF425QFG0CU&client_secret=CENIDR0OMZIIKAPI0CT145ZYU3J2OQUKFLRMRD4BLJSJRQZI&ll=40.7127281,-74.0060152&v=20180605&query=Dog Park&radius=4000&limit=100'

In [10]:
#use get function to see results

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cc095b36a60712126dc2553'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/dogrun_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1e5941735',
      'name': 'Dog Run',
      'pluralName': 'Dog Runs',
      'primary': True,
      'shortName': 'Dog Run'}],
    'hasPerk': False,
    'id': '4a6c8f18f964a520d0d01fe3',
    'location': {'address': 'Columbia Heights',
     'cc': 'US',
     'city': 'Brooklyn',
     'country': 'United States',
     'crossStreet': 'at Vine Street',
     'distance': 1630,
     'formattedAddress': ['Columbia Heights (at Vine Street)',
      'Brooklyn, NY 11201',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.701331040237875,
       'lng': -73.99387738397049}],
     'lat': 40.701331040237875,
     'lng': -73.99387738397049,
     'postalCode': '11201',
     'state': 'NY'},
    'name': 'Hillside Dog Park',
    'referr

In [11]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'pluralName': 'Dog Runs', 'name': 'Dog Run',...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a6c8f18f964a520d0d01fe3,Columbia Heights,...,"[Columbia Heights (at Vine Street), Brooklyn, ...","[{'lat': 40.701331040237875, 'label': 'display...",40.701331,-73.993877,NaN,11201,NY,Hillside Dog Park,v-1556125107,NaN
1,"[{'pluralName': 'Dog Runs', 'name': 'Dog Run',...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bd351c6462cb7130226de07,Warren St,...,"[Warren St (btwn Greenwich & West St), New Yor...","[{'lat': 40.71621653801577, 'label': 'display'...",40.716217,-74.012318,NaN,10007,NY,Warren Street Dog Park,v-1556125107,NaN
2,"[{'pluralName': 'Dog Runs', 'name': 'Dog Run',...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c5f498112e5c92851d31c4c,2nd Ave,...,"[2nd Ave (btwn E 15th & 17th St), New York, NY...","[{'lat': 40.73305819281746, 'label': 'display'...",40.733058,-73.983861,NaN,10003,NY,Stuyvesant Square Dog Park,v-1556125107,NaN
3,"[{'pluralName': 'Dog Runs', 'name': 'Dog Run',...",NaN,NaN,NaN,NaN,NaN,NaN,False,51b4ecac498eb9786be6de5b,310 W Broadway,...,"[310 W Broadway (Grand Street), New York, NY 1...","[{'lat': 40.722278918218436, 'label': 'display...",40.722279,-74.004236,NaN,10013,NY,Soho Grand Dog Park,v-1556125107,NaN
4,"[{'pluralName': 'Dog Runs', 'name': 'Dog Run',...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b54b023f964a52026c727e3,NaN,...,"[Brooklyn, NY 11231, United States]","[{'lat': 40.67980465275095, 'label': 'display'...",40.679805,-74.002987,NaN,11231,NY,Di Mattina Dog Park,v-1556125107,NaN


In [12]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Hillside Dog Park,Dog Run,Columbia Heights,US,Brooklyn,United States,at Vine Street,1630,"[Columbia Heights (at Vine Street), Brooklyn, ...","[{'lat': 40.701331040237875, 'label': 'display...",40.701331,-73.993877,NaN,11201,NY,4a6c8f18f964a520d0d01fe3
1,Warren Street Dog Park,Dog Run,Warren St,US,New York,United States,btwn Greenwich & West St,658,"[Warren St (btwn Greenwich & West St), New Yor...","[{'lat': 40.71621653801577, 'label': 'display'...",40.716217,-74.012318,NaN,10007,NY,4bd351c6462cb7130226de07
2,Stuyvesant Square Dog Park,Dog Run,2nd Ave,US,New York,United States,btwn E 15th & 17th St,2935,"[2nd Ave (btwn E 15th & 17th St), New York, NY...","[{'lat': 40.73305819281746, 'label': 'display'...",40.733058,-73.983861,NaN,10003,NY,4c5f498112e5c92851d31c4c
3,Soho Grand Dog Park,Dog Run,310 W Broadway,US,New York,United States,Grand Street,1073,"[310 W Broadway (Grand Street), New York, NY 1...","[{'lat': 40.722278918218436, 'label': 'display...",40.722279,-74.004236,NaN,10013,NY,51b4ecac498eb9786be6de5b
4,Di Mattina Dog Park,Dog Run,NaN,US,Brooklyn,United States,NaN,3673,"[Brooklyn, NY 11231, United States]","[{'lat': 40.67980465275095, 'label': 'display'...",40.679805,-74.002987,NaN,11231,NY,4b54b023f964a52026c727e3
5,McCarren Dog Park,Dog Run,Driggs Avenue,US,Brooklyn,United States,North 12th Street,4555,"[Driggs Avenue (North 12th Street), Brooklyn, ...","[{'lat': 40.720031425622324, 'label': 'display...",40.720031,-73.952889,NaN,11211,NY,4a9ff88df964a520c93d20e3
6,Fort Greene - Dog Park,Dog Run,NaN,US,Brooklyn,United States,NaN,3654,"[Brooklyn, NY 11201, United States]","[{'lat': 40.69045012098586, 'label': 'display'...",40.690450,-73.974213,NaN,11201,NY,4dc5415ed22dafda2f9f01f9
7,Church Square Dog Park,Dog Run,401 Willow Ave.,US,Hoboken,United States,4th & Willow,3995,"[401 Willow Ave. (4th & Willow), Hoboken, NJ 0...","[{'lat': 40.742203138352124, 'label': 'display...",40.742203,-74.033038,NaN,07030,NJ,4b93cf35f964a520c65234e3
8,dog park,Dog Run,Fdr,US,New York,United States,Maiden Ln.,966,"[Fdr (Maiden Ln.), New York, NY 10005, United ...","[{'lat': 40.70404559621692, 'label': 'display'...",40.704046,-74.005869,NaN,10005,NY,4fc55667e4b0205185aececf
9,Stuyvesant High School Dog Park,Dog Run,175 N End Ave,US,New York,United States,Warren St.,869,"[175 N End Ave (Warren St.), New York, NY 1028...","[{'lat': 40.71525408297036, 'label': 'display'...",40.715254,-74.015768,NaN,10282,NY,4b93f9caf964a520585e34e3


In [13]:
dataframe_filtered.name

0                      Hillside Dog Park
1                 Warren Street Dog Park
2             Stuyvesant Square Dog Park
3                    Soho Grand Dog Park
4                    Di Mattina Dog Park
5                      McCarren Dog Park
6                 Fort Greene - Dog Park
7                 Church Square Dog Park
8                               dog park
9        Stuyvesant High School Dog Park
10             South St Seaport Dog Park
11       Dog Park Under Manhattan Bridge
12          Tompkins Square Park Dog Run
13                     George's Dog Park
14                              Dog Park
15             Washington Square Dog Run
16                    380 Marin Dog Park
17                   East River Dog Park
18                        City Hall Park
19                  Battery Park Dog Run
20                         Star Dog Park
21                              Dog Park
22                              Dog Park
23                         Hope Dog Park
24              

In [14]:
#create a map of the dog park results

venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [15]:
from folium import plugins

dogpark_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(dogpark_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(incidents)

dogpark_map

In [16]:
venue_id = '4c5f498112e5c92851d31c4c' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4c5f498112e5c92851d31c4c?client_id=GKUAF244OPVYY01Z33OBZRVVQ5XFFLMKSKR5PNF425QFG0CU&client_secret=CENIDR0OMZIIKAPI0CT145ZYU3J2OQUKFLRMRD4BLJSJRQZI&v=20180605'

In [17]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['attributes', 'dislike', 'shortUrl', 'ratingColor', 'ok', 'timeZone', 'pageUpdates', 'name', 'tips', 'photos', 'contact', 'ratingSignals', 'hereNow', 'createdAt', 'stats', 'inbox', 'likes', 'beenHere', 'canonicalUrl', 'colors', 'page', 'location', 'rating', 'listed', 'verified', 'popular', 'bestPhoto', 'url', 'id', 'categories', 'specials', 'reasons'])


{'attributes': {'groups': []},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'bestPhoto': {'createdAt': 1383918353,
  'height': 640,
  'id': '527ceb11498e2052805eca14',
  'prefix': 'https://fastly.4sqi.net/img/general/',
  'source': {'name': 'Instagram', 'url': 'http://instagram.com'},
  'suffix': '/1174_Nr2GPQYH5nd1izIJqCrBhtEboWXlXysY8offK3ZoL5w.jpg',
  'visibility': 'public',
  'width': 640},
 'canonicalUrl': 'https://foursquare.com/v/stuyvesant-square-dog-park/4c5f498112e5c92851d31c4c',
 'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/dogrun_',
    'suffix': '.png'},
   'id': '4bf58dd8d48988d1e5941735',
   'name': 'Dog Run',
   'pluralName': 'Dog Runs',
   'primary': True,
   'shortName': 'Dog Run'}],
 'colors': {'algoVersion': 3,
  'highlightColor': {'photoId': '527ceb11498e2052805eca14',
   'value': -10463144},
  'highlightTextColor': {'photoId': '527ceb11498e2052805eca14', 'value': -

In [18]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.2


In [20]:
venue_id = '4a9ff88df964a520c93d20e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.3


In [21]:
venue_id = '4ba41754f964a5207a8038e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.2


In [22]:
venue_id = '51b4ecac498eb9786be6de5b' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.8


In [31]:
venue_id = '4fc55667e4b0205185aececf' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [32]:
venue_id = '4b93f9caf964a520585e34e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [33]:
venue_id = '4e11f28ed1647a0fff851926' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [34]:
venue_id = '4a68dff5f964a5202bcb1fe3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.3


In [35]:
venue_id = '52c471c9498e7c0e5acf11be' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [36]:
venue_id = '528ac52011d285bfc99d15fa' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [37]:
venue_id = '43768200f964a520472a1fe3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.4


In [38]:
venue_id = '3fd66200f964a520d8f11ee3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.2


In [39]:
venue_id = '4ba65f32f964a520614b39e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.6


In [40]:
venue_id = '4a678993f964a520a9c91fe3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [41]:
venue_id = '5b7b3a40464d65002cd7c0ff' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [42]:
venue_id = '4e0a48b46284cf220c358959' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.4


In [43]:
venue_id = '3fd66200f964a520def11ee3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.4


In [44]:
venue_id = '4df7b7adb0fbaeaaabe5d87e' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [45]:
venue_id = '4b51f714f964a520da5d27e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.5


In [46]:
venue_id = '4a88147ef964a5201e0520e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.3


In [47]:
venue_id = '40abf500f964a52035f31ee3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.6


In [48]:
venue_id = '4b871e4af964a5209db131e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [49]:
venue_id = '42daf100f964a52035261fe3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.4


In [50]:
venue_id = '3fd66200f964a520d7f11ee3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.7


In [51]:
venue_id = '4b723621f964a52018742de3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.8


In [52]:
venue_id = '40b68100f964a5207d001fe3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.6


In [53]:
venue_id = '49e7ce15f964a5201d651fe3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.0


In [54]:
venue_id = '4ba0e4e3f964a520498537e3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

9.5


In [55]:
venue_id = '4a15a2e8f964a520a9781fe3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.0


In [56]:
venue_id = '4a663f7cf964a52046c81fe3' 
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.3


In [24]:
#import a data table containing NYC neighborhoods and average rent prices

df = pd.read_html("https://www.rentcafe.com/average-rent-market-trends/us/ny/manhattan/")[2]
df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0))
df = df.reset_index(drop=True)

df

,Neighborhood,Average Rent
0,Marble Hill,"$1,694"
1,Inwood,"$2,225"
2,Washington Heights,"$2,243"
3,Randalls and Wards Islands,"$2,336"
4,East Harlem,"$3,334"
5,Tudor City,"$3,338"
6,Roosevelt Island,"$3,427"
7,Harlem,"$3,604"
8,Liberty Island,"$3,718"
9,Governors Island,"$3,718"


In [25]:
#make the average rent into a number rather than currency

df[df.columns[1:]] = df[df.columns[1:]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)
df.head()

,Neighborhood,Average Rent
0,Marble Hill,1694
1,Inwood,2225
2,Washington Heights,2243
3,Randalls and Wards Islands,2336
4,East Harlem,3334


In [26]:
from geopy.geocoders import Nominatim
nom=Nominatim()

In [27]:
#find the coordinates for each neighborhood

df["Coordinates"]=df["Neighborhood"].apply(nom.geocode)
df.head()

,Neighborhood,Average Rent,Coordinates
0,Marble Hill,1694,"(Marble Hill, New York County, NYC, New York, ..."
1,Inwood,2225,"(Inwood, Manhattan, Manhattan Community Board ..."
2,Washington Heights,2243,"(Washington Heights, Manhattan, Manhattan Comm..."
3,Randalls and Wards Islands,2336,"(Randalls and Wards Islands, Manhattan Communi..."
4,East Harlem,3334,"(East Harlem, Manhattan, Manhattan Community B..."


In [28]:
#add in new columns with lat and lng for each neighborhood

df["Latitude"]=df["Coordinates"].apply(lambda x: x.latitude if x != None else None)
df["Longitude"]=df["Coordinates"].apply(lambda x: x.longitude if x != None else None)
df.head()

,Neighborhood,Average Rent,Coordinates,Latitude,Longitude
0,Marble Hill,1694,"(Marble Hill, New York County, NYC, New York, ...",40.876298,-73.910429
1,Inwood,2225,"(Inwood, Manhattan, Manhattan Community Board ...",40.869258,-73.920495
2,Washington Heights,2243,"(Washington Heights, Manhattan, Manhattan Comm...",40.840198,-73.940221
3,Randalls and Wards Islands,2336,"(Randalls and Wards Islands, Manhattan Communi...",40.791448,-73.921024
4,East Harlem,3334,"(East Harlem, Manhattan, Manhattan Community B...",40.794722,-73.942500


In [29]:
#drop any NaN values - there was only one in this data set

df=df.dropna()

In [30]:
#create a folium map with markers that represent different rent prices - orange is higher and larger circles represent higher prices

rent_map = folium.Map(location = [latitude, longitude], zoom_start = 13)

for index, row in df.iterrows():
    radius = (row["Average Rent"]/150)
    if (row["Average Rent"]>4500):
        color="#E37222" # tangerine
    else:
        color="#0A8A9F" # teal
    folium.CircleMarker(location=(row["Latitude"],
                                  row["Longitude"]),
                        radius=radius,
                        color=color,
                        fill=True).add_to(rent_map)

rent_map